# [자연어처리]
# 12주차(12-2). BERT와 WordPiece

# 202002961 김현주
* **모든 셀 실행** 후 제출하시기 바랍니다.
* **실습 (12-2-1)**이 있습니다.

> ### 중간 중간 **'### ... 이 부분을 완성하시오'** 라는 부분의 코드를 완성해 보기 바랍니다. (강의 자료를 참고하여 '???' 부분을 작성해 보시기 바랍니다.)

---

In [ ]:
! pip install transformers ### ... 이 부분을 완성하시오

In [ ]:
import pandas as pd
from transformers import BertTokenizer ### ... 이 부분을 완성하시오

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
result = tokenizer.tokenize('Here is the sentence I want embeddings for .') ### ... 이 부분을 완성하시오
print(result)

['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.']


In [ ]:
print(tokenizer.vocab['here']) ### ... 이 부분을 완성하시오

2182


# 아래의 셀을 실행하면 오류가 날 것임 (KeyError: 'embeddings')
- vocabulary (vocab) 안에 embeddings라는 단어가 없기 때문
- embeddings는 em, ##bed, ##ding, ##s로 쪼개어짐

# 그 다음 셀부터 다시 실행하면 됨

In [ ]:
print(tokenizer.vocab['embeddings'])

KeyError: 'embeddings'

# 아래의 셀부터 다시 실행

In [ ]:
with open('vocabulary.txt', 'w') as f:
  for token in tokenizer.vocab.keys():
    f.write(token + '\n')

In [ ]:
df = pd.read_fwf('vocabulary.txt', header=None)
df

,0
0,[PAD]
1,[unused0]
2,[unused1]
3,[unused2]
4,[unused3]
...,...
30517,##．
30518,##／
30519,##：
30520,##？


In [ ]:
print('단어 집합의 크기 :', len(df))

단어 집합의 크기 : 30522


In [ ]:
df.loc[4667].values[0]

'##ding'

In [ ]:
df.loc[102].values[0]

'[SEP]'

---

# **[[실습 12-2-1]]** WordPiece와 다른 토크나이저 (BPE)를 비교하는 실습
- **BERT**는 **WordPiece**라는 방식으로 토큰화(tokenize)를 하고, **GPT**는 **BPE (Byte Pair Encoding)**이라는 방식으로 토큰화를 함.
- 동일한 텍스트에 대해 WordPiece, BPE를 사용해 토큰화해보고, 각 토크나이저의 결과를 비교하고 차이점을 분석해보자!

## **WordPiece 토크나이저**
- transformers 라이브러리의 BertTokenizer 사용

In [ ]:
from transformers import BertTokenizer

# WordPiece Tokenizer
wordpiece_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# text 변수에 새로운 예시 문장을 만들어보시오.

text = "Natural language processing is interesting." ## 토큰화 결과를 확인하고 싶은 문장을 만들어 보시오.

wordpiece_tokens = wordpiece_tokenizer.tokenize(text)
print("WordPiece Tokens:", wordpiece_tokens)

WordPiece Tokens: ['natural', 'language', 'processing', 'is', 'interesting', '.']


## **BPE 토크나이저**
- transformers 라이브러리의 GPT2Tokenizer 사용

In [ ]:
from transformers import GPT2Tokenizer

# BPE Tokenizer
bpe_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
bpe_tokens = bpe_tokenizer.tokenize(text)
print("BPE Tokens:", bpe_tokens)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

BPE Tokens: ['Natural', 'Ġlanguage', 'Ġprocessing', 'Ġis', 'Ġinteresting', '.']


---

## **비교 분석**
- 아래의 3가지 관점에서 WordPiece와 BPE의 결과를 비교해보자

### WordPiece (BERT) vs BPE (GPT)
1. **구분자 모양**: 각 토큰의 경계를 나타내는 구분자 확인
2. **토큰 길이**: 각 토크나이저가 생성한 토큰 수 비교
3. **희귀 단어 토큰화**: 희귀 단어(rare word)나 새로운 단어(Out-of-vocabulary, OOV)가 어떻게 처리되는지 확인



### **1. 구분자 모양**
- 각 토큰의 경계를 나타내는 구분자 확인
- BERT는 ##을 사용한다. GPT는 무엇을 구분자로 사용할까?

In [ ]:
print(f"Original Text: {text}")
print(f"WordPiece Tokens: {wordpiece_tokens}")
print(f"BPE Tokens: {bpe_tokens}")

Original Text: Natural language processing is interesting.
WordPiece Tokens: ['natural', 'language', 'processing', 'is', 'interesting', '.']
BPE Tokens: ['Natural', 'Ġlanguage', 'Ġprocessing', 'Ġis', 'Ġinteresting', '.']


### **2. 토큰 길이**
- 각 토크나이저가 생성한 토큰 수 비교
- 토크나이저(WordPiece vs BPE)의 차이에 따른 토큰화 결과 길이 비교

In [ ]:
print(f"WordPiece Token Lengths: {len(wordpiece_tokens)}")
print(f"BPE Token Lengths: {len(bpe_tokens)}")

WordPiece Token Lengths: 6
BPE Token Lengths: 6


### **3. 희귀 단어 토큰화**
- 희귀 단어(rare word)나 새로운 단어(Out-of-vocabulary, OOV)가 어떻게 처리되는지 확인
- 자주 사용되지 않는 단어를 포함한 문장을 만들어 확인해보자!

In [ ]:
# rare_text 변수에 새로운 예시 문장을 만들어보시오.

rare_text = '"BRAT", the album by Charli XCX, is the best album of the year.' # 자주 사용되지 않는 단어를 포함한 문장을 만들어 확인해보자!

print("WordPiece:", wordpiece_tokenizer.tokenize(rare_text))
print("BPE:", bpe_tokenizer.tokenize(rare_text))

WordPiece: ['"', 'brat', '"', ',', 'the', 'album', 'by', 'char', '##li', 'x', '##c', '##x', ',', 'is', 'the', 'best', 'album', 'of', 'the', 'year', '.']
BPE: ['"', 'BR', 'AT', '",', 'Ġthe', 'Ġalbum', 'Ġby', 'ĠChar', 'li', 'ĠX', 'C', 'X', ',', 'Ġis', 'Ġthe', 'Ġbest', 'Ġalbum', 'Ġof', 'Ġthe', 'Ġyear', '.']


## **결과 분석**
- 예) 위의 토큰 길이, 구분자, 희귀 단어 3가지 관점으로 살펴본 비교 분석에 따르면 WordPiece는 어떤 특징이 있는 것 같고, BPE는 어떤 특징이 있는 것 같다.

WordPiece와 BPE 두 방법은 비슷한 길이의 토큰을 생성하지만, 토큰화 방식에 차이를 보였습니다.
WordPiece는 ## 구분자를 사용하여 단어의 중간부터 등장하는 서브워드라는 것을 알려주고, 대문자를 소문자로 통일하여 일관성을 유지합니다.
BPE는 Ġ 구분자를 사용하여 공백 다음에 오는 단어의 시작을 나타내고, 대문자와 소문자를 구별하여 희귀 단어도 대문자를 유지할 수 있습니다.


---

# [파일] -> [다운로드] -> [.ipynb 다운로드]